<a href="https://colab.research.google.com/github/sachini786/Employee-Management-System/blob/main/Tag_Recommendation_TFIDFwithCosine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#authenticate with Google
from google.colab import auth
import pandas as pd
import numpy as np
auth.authenticate_user()

In [ ]:
# access and query a BigQuery dataset
%%bigquery df --project fyp-sachini
SELECT id, title, body, tags
FROM `bigquery-public-data.stackoverflow.posts_questions`
ORDER BY creation_date DESC
LIMIT 50000

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
temp_df=df.copy()

In [ ]:
#extract code snippets from the body field
import re

from bs4 import BeautifulSoup
def extract_and_remove_code_blocks(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    code_blocks = []
    for code in soup.find_all('code'):
        if code is not None and code.get_text(strip=True):
            code_blocks.append(str(code))  # Convert tag to string immediately
            code.decompose()

    cleaned_html = str(soup)
    code_blocks = ' '.join(code_blocks)
    return cleaned_html, code_blocks
temp_df[['body', 'code']] = temp_df['body'].apply(extract_and_remove_code_blocks).apply(pd.Series)

temp_df


,id,title,body,tags,code
0,73842327,Unable to start ypserv in Ubuntu,<p>I was trying to configure NIS master Server...,linux|ubuntu|linux-kernel|nis,
1,73842326,BOTH WINDOWS POWERSHELL / ISE HAS STOPPED WORK...,<p>Description:\nStopped working</p>\n<p>Probl...,powershell|visual-studio-code,
2,73842325,Migrating Java 8 project to Java 17,<p>We want to migrate our microservices which ...,java-8|migration|upgrade|java-17,
3,73842324,I'm trying to implement quickselect by partiti...,<p>Here's the java code</p>\n<p>The select fun...,java|quickselect,<code>public static Comparable select(Comparab...
4,73842323,Invalid request (Unsupported SSL request) - La...,<p>Microsoft Windows 11 Pro Build 22000\nPHP 8...,laravel|ssl,<code>D:\Programming\XAMPP\xampp\htdocs\shoppi...
...,...,...,...,...,...
49995,73732396,Flutter Firestore Pagination of a Stream with ...,<p>I've been working on a chat feature in my a...,flutter|dart|google-cloud-firestore|flutter-st...,<code>pageNumber</code> <code>getMessages</cod...
49996,73732393,Combine excel files,<p>Can someone help how to get output in excel...,python|excel|pandas|export-to-excel,<code>import pandas as pd\nimport os\ninput_pa...
49997,73732392,How to Pass The Value of a TextView to A Spinner?,"<p>I have a TextView with the value ""Dog"" whic...",android|kotlin|textview|sharedpreferences|spinner,
49998,73732390,Spring Integration (java dsl) ERR_INVALID_CHUN...,<p>So i want to provide a http-endpoint in Spr...,java|spring|http|spring-integration|spring-int...,<code> return IntegrationFlows.from(Htt...


In [ ]:
#extracting "stack trace–like" entries from the code column
from bs4 import BeautifulSoup

def extract_stack_trace(html):
    soup = BeautifulSoup(html, 'html.parser')
    filtered_code = []
    for code_tag in soup.find_all('code'):
        code_text = code_tag.text
        if any(ext in code_text for ext in (".py", ".java", ".js", ".cs", ".cpp", ".go", ".rs", ".dart")):
            filtered_code.append(code_text)
            code_tag.decompose()  # Remove this code tag from soup
    return  str(soup), "\n".join(filtered_code)
temp_df[['code', 'stack_trace']] = temp_df['code'].apply(extract_stack_trace).apply(pd.Series)
temp_df['tags'] = temp_df['tags'].apply(lambda x: x.split('|'))


In [ ]:
temp_df.replace('', np.nan, inplace=True)

In [ ]:
temp_df.dropna(subset=['stack_trace'], inplace=True)

In [ ]:
import re
from bs4 import BeautifulSoup
def clean_text(text):
    text = text.lower()  # ✅ Lowercasing
    text = BeautifulSoup(text, "html.parser").get_text()  # ✅ Remove HTML tags
    text = text.replace('.', ' ')  # ✅ Optional: Break sentences into words
    text = re.sub(r'\s+', ' ', text).strip()  # ✅ Normalize whitespace
    return text


In [ ]:
temp_df['clean_title'] = temp_df['title'].apply(clean_text)
temp_df['clean_body'] = temp_df['body'].apply(clean_text)

/tmp/ipython-input-8-4134714940.py:5: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a filename than HTML or XML.

If you meant to use Beautiful Soup to parse the contents of a file on disk, then something has gone wrong. You should open the file first, using code like this:

    filehandle = open(your filename)

You can then feed the open filehandle into Beautiful Soup instead of using the filename.

However, if you want to parse some data that happens to look like a filename, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  text = BeautifulSoup(text, "html.parser").get_text()  # ✅ Remove HTML tags


In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load("en_core_web_sm")

def advanced_clean(text):
    if pd.isna(text):
        return ""
    doc = nlp(text)
    lemmatized = [token.lemma_ for token in doc if token.is_alpha and token.text not in STOP_WORDS]
    return ' '.join(lemmatized)



In [ ]:
# Apply to clean text columns only
temp_df['lemma_title'] = temp_df['clean_title'].apply(advanced_clean)

In [ ]:
temp_df['lemma_body'] = temp_df['clean_body'].apply(advanced_clean)

In [ ]:
from bs4 import BeautifulSoup

def clean_code(code):
    if pd.isna(code):  # Handle NaN values safely
        return ""
    code = BeautifulSoup(str(code), "html.parser").get_text()
    return code.strip()


In [ ]:
temp_df['clean_code'] = temp_df['code'].apply(clean_code)

In [ ]:
def clean_stack_trace(trace):
    trace = BeautifulSoup(trace, "html.parser").get_text()
    trace = re.sub(r'\s+', ' ', trace)
    # Optional: remove long file paths
    trace = re.sub(r'[A-Za-z]:\\[^ ]+|/[^ ]+', '', trace)
    return trace.strip()


In [ ]:
temp_df['clean_stack_trace'] = temp_df['stack_trace'].apply(clean_code)

/tmp/ipython-input-13-3975822801.py:6: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  code = BeautifulSoup(str(code), "html.parser").get_text()
/tmp/ipython-input-13-3975822801.py:6: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack

In [ ]:
# Fill missing values
for col in ['lemma_title', 'lemma_body', 'clean_code', 'clean_stack_trace']:
    temp_df[col] = temp_df[col].fillna('')

In [ ]:
tfidf_title = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
tfidf_text = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
tfidf_code = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
tfidf_stack = TfidfVectorizer(max_features=3000, ngram_range=(1, 2))

In [ ]:
X_title = tfidf_title.fit_transform(temp_df['lemma_title'])
X_text = tfidf_text.fit_transform(temp_df['lemma_body'])
X_code = tfidf_code.fit_transform(temp_df['clean_code'])
X_stack = tfidf_stack.fit_transform(temp_df['clean_stack_trace'])


In [ ]:
X = hstack([X_title, X_text, X_code, X_stack])

In [ ]:
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(temp_df['tags'])

In [ ]:
# Ensure X and Y are the same length
min_len = min(X.shape[0], Y.shape[0])
X = X[:min_len]
Y = Y[:min_len]

# Compute tag centroids
tag_centroids = {}
for i, tag in enumerate(mlb.classes_):
    tag_docs_idx = np.where(Y[:, i] == 1)[0]
    if len(tag_docs_idx) > 0:
        tag_vector = X[tag_docs_idx].mean(axis=0)
        tag_centroids[tag] = tag_vector

In [ ]:
def recommend_tags_cosine_separate(row, top_k=5):
    vec_title = tfidf_title.transform([row["lemma_title"]])
    vec_text = tfidf_text.transform([row["lemma_body"]])
    vec_code = tfidf_code.transform([row["clean_code"]])
    vec_stack = tfidf_stack.transform([row["clean_stack_trace"]])

    vec = hstack([vec_title, vec_text, vec_code, vec_stack])
    vec_dense = np.asarray(vec.todense())  # <- fix

    similarities = []
    for tag, centroid in tag_centroids.items():
        centroid_dense = np.asarray(centroid)
        sim = cosine_similarity(vec_dense, centroid_dense).flatten()[0]
        similarities.append((tag, sim))

    top_tags = sorted(similarities, key=lambda x: x[1], reverse=True)[:top_k]
    return [tag for tag, _ in top_tags]

In [ ]:
def evaluate_recommender_cosine_separate(df, top_k=5):
    total_precision = 0
    total_recall = 0
    total_samples = 0

    for _, row in df.iterrows():
        true_tags = row['tags']
        if not true_tags:
            continue

        predicted_tags = recommend_tags_cosine_separate(
            row, top_k=top_k
        )



        hits = len(set(predicted_tags) & set(true_tags))
        precision = hits / top_k
        recall = hits / len(true_tags)

        total_precision += precision
        total_recall += recall
        total_samples += 1

    avg_precision = total_precision / total_samples
    avg_recall = total_recall / total_samples
    print(f"Precision@{top_k}: {avg_precision:.4f}")
    print(f"Recall@{top_k}: {avg_recall:.4f}")


In [ ]:
evaluate_recommender_cosine_separate(temp_df.sample(100), top_k=5)

Precision@5: 0.5140
Recall@5: 0.7907


In [ ]:
def compute_tag_centroids_without_stacktrace():
    vec_title = tfidf_title.transform(temp_df['lemma_title'])
    vec_text = tfidf_text.transform(temp_df["lemma_body"])
    vec_code = tfidf_code.transform(temp_df["clean_code"])
    vec = hstack([vec_title, vec_text, vec_code])

    centroids = {}
    for i, tag in enumerate(mlb.classes_):
        tag_indices = np.where(Y[:, i] == 1)[0]
        if len(tag_indices) > 0:
            tag_vecs = vec[tag_indices]
            centroids[tag] = np.asarray(tag_vecs.mean(axis=0))
    return centroids

In [ ]:
tag_centroids_without_stacktrace = compute_tag_centroids_without_stacktrace()

In [ ]:
def recommend_tags_cosine_separate_without_stacktrace(row, top_k=5):
    vec_title = tfidf_title.transform([row["lemma_title"]])
    vec_text = tfidf_text.transform([row["lemma_body"]])
    vec_code = tfidf_code.transform([row["clean_code"]])

    vec = hstack([vec_title, vec_text, vec_code])
    vec_dense = np.asarray(vec.todense())

    similarities = []
    for tag, centroid in tag_centroids_without_stacktrace.items():
        centroid_dense = np.asarray(centroid)
        sim = cosine_similarity(vec_dense, centroid_dense).flatten()[0]
        similarities.append((tag, sim))

    top_tags = sorted(similarities, key=lambda x: x[1], reverse=True)[:top_k]
    return [tag for tag, _ in top_tags]

In [ ]:
def evaluate_recommender_cosine_separate_without_stacktrace(df, top_k=5):
    total_precision = 0
    total_recall = 0
    total_samples = 0

    for _, row in df.iterrows():
        true_tags = row['tags']
        if not true_tags:
            continue

        predicted_tags = recommend_tags_cosine_separate_without_stacktrace(
            row, top_k=top_k
        )

        hits = len(set(predicted_tags) & set(true_tags))
        precision = hits / top_k
        recall = hits / len(true_tags)

        total_precision += precision
        total_recall += recall
        total_samples += 1

    avg_precision = total_precision / total_samples
    avg_recall = total_recall / total_samples
    print(f"Precision@{top_k}: {avg_precision:.4f}")
    print(f"Recall@{top_k}: {avg_recall:.4f}")


In [ ]:
evaluate_recommender_cosine_separate_without_stacktrace(temp_df.sample(100), top_k=5)

Precision@5: 0.5040
Recall@5: 0.7337
